# BB84 Protocol Components

## Core Concepts

### Quantum Elements
- **Qubit:** The basic unit of quantum information, can be 0, 1, or a superposition of both
- **Quantum Circuit:** A sequence of quantum operations (gates) applied to qubits
- **Basis States:**
  - *Z Basis (Computational):* States |0⟩ and |1⟩
  - *X Basis (Hadamard):* States |+⟩ and |−⟩ (superpositions of |0⟩ and |1⟩)

### Quantum Gates
- **Hadamard Gate (H):** Rotates between the Z and X bases
- **X Gate (Pauli-X):** Flips the state of a qubit (quantum NOT gate)
- **Measure Gate:** Collapses a qubit's superposition to yield a classical bit

### Protocol Participants
- **Alice:** The sender in the BB84 protocol
- **Bob:** The receiver in the BB84 protocol

### Technical Components
- **Qiskit:** Open-source SDK for working with quantum computers
- **AerSimulator:** High-performance quantum circuit simulator
- **NumPy:** Python library for numerical operations
- **Classical Bit:** Standard binary unit (0 or 1)

### Protocol Terminology
- **Transpile:** Process of optimizing quantum circuits for specific backends
- **Shot:** Single execution of a quantum circuit
- **Counts:** Results showing measurement outcome frequencies
- **Basis Match:** When Alice and Bob choose the same measurement basis
- **Bit Match:** Matching bits between Alice and Bob when bases align

In [ ]:
import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator

# -=-=-=- Simulation Setup -=-=-=-
# Start with one qubit
num_qubits = 1
simulator = AerSimulator()

# -=-=-=- Alice's Choices -=-=-=-
# Alice's choices of basis and bit (Randomly chosen)
alice_bit = np.random.randint(2, size=num_qubits)
alice_basis = np.random.randint(2, size=num_qubits)

print(f"Alice's random bit: {alice_bit}")
print(f"Alice's chosen basis: {'Z' if alice_basis == 0 else 'X'}")

# -=-=-=- Alice's Qubit Prep -=-=-=-
qc_alice_prep = QuantumCircuit(num_qubits)
if alice_bit == 1:
    qc_alice_prep.x(0)  # Apply X gate if Alice's bit is 1 (Start with |1>)
if alice_basis == 1:
    qc_alice_prep.h(0)  # Apply H gate if Alice's basis is 1 (Start with |+>)
    
# -=-=-=- Bob's Choices -=-=-=-
# Bob's choices of basis (Randomly chosen)
qc_bob_measure = qc_alice_prep.copy()
bob_basis = np.random.randint(2, size=num_qubits)

print(f"Bob's chosen basis: {'Z' if bob_basis == 0 else 'X'}")

# -=-=-=- Bob's Measurement -=-=-=-
qc_bob_measure = qc_alice_prep.copy()
if bob_basis == 1:
    qc_bob_measure.h(0)  # Apply H gate if Bob's basis is 1 (Measure in |+> basis)
    
# Measure the qubit, representing the result in classical bits
qc_bob_measure.measure_all()

# -=-=-=- Simulation Execution -=-=-=-
# Transpile the circuit for the simulator
transpiled_qc = transpile(qc_bob_measure, simulator)

# Run
job = simulator.run(transpiled_qc, shots=1)
result = job.result()
counts = result.get_counts(transpiled_qc)

# Get and print Bob's measured bit
bob_measured_bit = int(list(counts.keys())[0])
print(f"Bob's measured bit: {bob_measured_bit}")

# -=-=- Check if Bases Match and Compare Bits -=-=-
bases_match = (alice_basis == bob_basis)
print(f"Bases match:{bases_match}")

if bases_match:
    bits_match = (alice_bit == bob_measured_bit)
    print(f"Bits match:{bits_match}")
    if bits_match:
        print("Qubit successfully transmitted and bases matched (no noise or eavesdropping simulated yet).")
    else:
         print("Bits don't match even though bases matched, indicating a potential eavesdropping or noise.")
else:
    print("Bases did not match. This round is discarded in the sifting stage.")

## Simulating Multiple Rounds and Basis Sifting

### Key Components
- **Raw Key:** The sequence of bits that Alice and Bob keep after comparing their basis choices and discarding the rounds where the bases did not match. This key still needs further processing (error correction and privacy amplification).

- **Basis Sifting:** The process in BB84 where Alice and Bob publicly compare their chosen bases for each qubit transmission and discard the rounds where their bases differ.

### Technical Elements
- **Num_qubits:** The total number of qubits sent in the simulation (represents the number of rounds)
- **qc_bb84:** The single Qiskit QuantumCircuit representing the combined operations for all the qubit transmissions in the simulation
- **Barrier:** A visual separator in a quantum circuit, does not perform a quantum operation but can affect transpilation

### Data Structures
- **Matching Bases Indices:** A list of the indices (which qubit number) for the rounds where Alice's and Bob's bases matched
- **Sifted Alice Bits:** The list of Alice's original bits from the rounds where the bases matched
- **Sifted Bob Bits:** The list of Bob's measured bits from the rounds where the bases matched

In [ ]:
import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.result import Counts
from IPython.display import display

# -=-=-=- Simulation Setup -=-=-=-
num_qubits = 27
simulator = AerSimulator()

# -=-=-=- Alice's Choices -=-=-=-
# Alice's choices of basis and bit (Randomly chosen)
alice_bits = [int(x) for x in np.random.randint(0, 2, num_qubits)]
alice_bases = [int(x) for x in np.random.randint(0, 2, num_qubits)]

print(f"Alice's random bits (first 10): {alice_bits[:10]}")
print(f"Alice's chosen bases (first 10): {['Z' if b == 0 else 'X' for b in alice_bases[:10]]}")

# -=-=-=- Bob's Choices -=-=-=-
# Bob's choices of basis (Randomly chosen)
bob_bases = np.random.randint(0, 2, num_qubits) # 0: Z, 1: X

print(f"Bob's chosen bases (first 10): {['Z' if b == 0 else 'X' for b in bob_bases[:10]]}")

# -=-=-=- Simulation Execution -=-=-=-
# Transpile the circuit for the simulator
qc_bb84 = QuantumCircuit(num_qubits, num_qubits) # num_qubits quantum, num_qubits classical

for i in range(num_qubits):
    # Alice's preparation
    if alice_bits[i] == 1:
        qc_bb84.x(i)
    if alice_bases[i] == 1:
        qc_bb84.h(i)
        
    # Add a barrier for visual separation (optional)
    qc_bb84.barrier(i)
    
    # Bob's measurement
    if bob_bases[i] == 1:
        qc_bb84.h(i)

    # Measure the qubit, representing the result in classical bits
    qc_bb84.measure(i, i)
    
# -=-=-=- Run -=-=-=-
transpiled_qc = transpile(qc_bb84, simulator)

shots = 1 # For basis sifting, we essentially do 1 'logical' run of sending all qubits
job = simulator.run(transpiled_qc, shots=shots)
result = job.result()
counts = result.get_counts(transpiled_qc)

# Get Bob's measured bits (this will be a dictionary with one key)
bob_measured_bits_str = list(counts.keys())[0]
# Convert the string of bits to a list of integers (reversed order from Qiskit output)
bob_measured_bits = [int(b) for b in bob_measured_bits_str][::-1]

# -=-=-=- Basis Sifting -=-=-=-
sifted_alice_bits = []
sifted_bob_bits = []
matching_bases_indices = []

for i in range(num_qubits):
    if alice_bases[i] == bob_bases[i]:
        # Bases match, keep this bit for the raw key
        sifted_alice_bits.append(alice_bits[i])
        sifted_bob_bits.append(bob_measured_bits[i])
        matching_bases_indices.append(i)

print(f"\nNumber of qubits sent: {num_qubits}")
print(f"Number of bits after sifting: {len(sifted_alice_bits)}")

# The 'sifted_alice_bits' and 'sifted_bob_bits' lists now form the raw key
# before error estimation, error correction, and privacy amplification.

# Print first 10 bits of the raw key
if len(sifted_alice_bits) > 0:
    print(f"Alice's raw key (first 10): {sifted_alice_bits[:10]}")
    print(f"Bob's raw key (first 10):   {sifted_bob_bits[:10]}")
    
# -=-=-=- Visualization Example -=-=-=-
# Create a circuit for visualization
qc_vis = QuantumCircuit(num_qubits, num_qubits)

for i in range(num_qubits):
    # Alice's preparation - matches qc_bb84
    if alice_bits[i] == 1:
        qc_vis.x(i)
    if alice_bases[i] == 1:
        qc_vis.h(i)
        
    # Add a barrier for visual separation
    qc_vis.barrier(i)
    
    # Bob's measurement - matches qc_bb84
    if bob_bases[i] == 1:
        qc_vis.h(i)
    
    # Measure the qubit
    qc_vis.measure(i, i)

# Display the circuit with customized style
display(qc_vis.draw(output='mpl', style={'backgroundcolor': '#FFFFFF'}, fold=40))

In this example above, when Alice's and Bob's raw key bits have matching bases, it is confirmed that the quantum information was successfully transmitted for those specific qubits.

## Incorporating Depolarizing Noise and Calculating QBER

### Noise Components
- **Depolarizing Noise:** A type of noise that randomly flips a qubit's state (with some probability), making it less correlated with the original state. A common model for errors in quantum channels.
- **Noise Model:** A representation of the errors that occur on a real or simulated quantum device or channel. Used by the simulator to mimic non-ideal behavior.
- **p_depol:** The parameter representing the probability of depolarization occurring.

### Implementation Elements
- **depolarizing_error:** A specific function in Qiskit Aer's noise module to create a depolarizing error object.
- **add_all_qubit_quantum_error:** A method to add a specific error type to the noise model, applying it to certain gates or all qubits when those gates are used.
- **Identity Gate (id):** A quantum gate that does nothing to the qubit's state. Used in this simulation as a placeholder where the noise model applies the depolarizing error, representing the noisy quantum channel.

### Error Metrics
- **QBER (Quantum Bit Error Rate):** The percentage or fraction of bits that are different between Alice's and Bob's raw keys in the rounds where their bases matched. A key indicator of noise or eavesdropping.
- **Errors:** The count of bits that differ between Alice's and Bob's raw keys after sifting.
- **Sifted Bits Count:** The total number of bits in the raw key after sifting.

### Measurement Statistics
- **Qubit Counts:** Data structure used in the code to aggregate the measurement outcomes for each individual qubit across all simulation shots.
- **Most Frequent Outcome:** The measurement outcome (0 or 1) that occurred most often for a specific qubit across all simulation shots.

### Security Analysis
- **Secure Key Rate (Asymptotic):** A theoretical formula that estimates the number of secure bits that can be distilled per sifted bit, based on the QBER. Applies in the limit of sending a very large number of qubits.
- **Binary Entropy Function (h(x)):** A mathematical function used in the secure key rate formula to quantify the uncertainty associated with a probability.
- **Estimated Secure Key Length:** The calculated length of the final secure key based on the estimated secure key rate and the number of sifted bits.
- **QBER Threshold:** The maximum QBER (approximately 11% for BB84) above which it is generally not possible to distill a secure key.

In [ ]:
import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.result import Counts
from IPython.display import display
from qiskit_aer.noise import NoiseModel, depolarizing_error


# -=-=-=- Simulation Setup -=-=-=-
num_qubits = 27
simulator = AerSimulator()

# -=-=-=- Noise Model Setup -=-=-=-
# List of depolarizing probabilities to test
p_depol_values = np.arange(0, 0.21, 0.02)  # [0, 0.02, 0.04, ..., 0.2]
qber_results = []  # Store QBER for each p_depol value

print("Testing BB84 protocol with different noise levels:")
for p_depol in p_depol_values:
    print(f"\n-=-=-=- Testing with p_depol = {p_depol:.2f} -=-=-=-")
    
    # Create noise model for current p_depol
    noise_model = NoiseModel()
    depol_error = depolarizing_error(p_depol, 1)
    noise_model.add_all_qubit_quantum_error(depol_error, ['id'])
   
    print(f"\nTesting with depolarizing probability: {p_depol:.3f}")
    
    # -=-=-=- Alice and Bob's Choices -=-=-=-
    alice_bits = np.random.randint(0, 2, num_qubits)
    alice_bases = np.random.randint(0, 2, num_qubits)
    bob_bases = np.random.randint(0, 2, num_qubits)
    
    # -=-=-=- Create Circuit -=-=-=-
    qc_bb84_noisy = QuantumCircuit(num_qubits, num_qubits)
    
    # Prepare all qubits
    for i in range(num_qubits):
        if alice_bits[i] == 1:
            qc_bb84_noisy.x(i)
        if alice_bases[i] == 1:
            qc_bb84_noisy.h(i)
        qc_bb84_noisy.id(i)  # Apply identity to include noise
        if bob_bases[i] == 1:
            qc_bb84_noisy.h(i)
        qc_bb84_noisy.measure(i, i)
    
    # -=-=-=- Run Circuit -=-=-=-
    transpiled_qc = transpile(qc_bb84_noisy, simulator)
    job = simulator.run(transpiled_qc, noise_model=noise_model, shots=1000)
    result = job.result()
    counts = result.get_counts(transpiled_qc)
    
    # Process results
    qubit_counts = [{} for _ in range(num_qubits)]
    for bitstring, count in counts.items():
        reversed_bitstring = bitstring[::-1]
        for i in range(num_qubits):
            outcome = int(reversed_bitstring[i])
            if outcome not in qubit_counts[i]:
                qubit_counts[i][outcome] = 0
            qubit_counts[i][outcome] += count
    
    # Calculate QBER
    matching_bases_indices = []
    errors = 0
    sifted_bits_cnt = 0
    
    for i in range(num_qubits):
        if alice_bases[i] == bob_bases[i]:
            sifted_bits_cnt += 1
            matching_bases_indices.append(i)
            measured_counts = qubit_counts[i]  # Get counts for this specific qubit
            
            if 0 in measured_counts and 1 in measured_counts:
                most_frequent_outcome = 0 if measured_counts[0] > measured_counts[1] else 1
            elif 0 in measured_counts:
                most_frequent_outcome = 0
            elif 1 in measured_counts:
                most_frequent_outcome = 1
            else:
                continue
                
            if alice_bits[i] != most_frequent_outcome:
                errors += 1
    
    # Calculate final metrics for this p_depol value
    if sifted_bits_cnt > 0:
        qber = errors / sifted_bits_cnt
        qber_results.append(qber)  # Store QBER for this p_depol value    # Create noise model for current p_depol
    noise_model = NoiseModel()
    
    depol_error = depolarizing_error(p_depol, 1) # 1-qubit depolarizing error
    
    noise_model.add_all_qubit_quantum_error(depol_error, ['id'])

    print(f"\nTesting with depolarizing probability: {p_depol:.3f}")
    
    # -=-=-=- Alice Generates Random Bits and Bases -=-=-=-
    alice_bits = np.random.randint(0, 2, num_qubits)
    alice_bases = np.random.randint(0, 2, num_qubits) # 0: Z, 1: X

    # -=-=-=- Bob Generates Random Bases -=-=-=-
    bob_bases = np.random.randint(0, 2, num_qubits) # 0: Z, 1: X
    
    # -=-=-=- Simulation of Transmission with Noise -=-=-=-
    qc_bb84_noisy = QuantumCircuit(num_qubits, num_qubits) # num_qubits quantum, num_qubits classical
    for i in range(num_qubits):
        if alice_bits[i] == 1:
            qc_bb84_noisy.x(i)
        if alice_bases[i] == 1:
            qc_bb84_noisy.h(i)
            
        qc_bb84_noisy.id(i)  # Apply identity to include noise model
        
        # Bob's measurement rotation
        if bob_bases[i] == 1:
            qc_bb84_noisy.h(i)
            
        # Measure the qubit, representing the result in classical bits
        qc_bb84_noisy.measure(i, i)
    
        # -=-=-=- Run with Noise Model -=-=-=-
        transpiled_qc = transpile(qc_bb84_noisy, simulator)
        shots = 1000 # Number of times to run the entire circuit (number of full BB84 runs in simulation).
        job = simulator.run(transpiled_qc, noise_model=noise_model, shots=shots)
        result = job.result()
        counts = result.get_counts(transpiled_qc)  
        
        qubit_counts = [{} for _ in range(num_qubits)] # Initialize a list of dictionaries to store counts for each qubit.
        for bitstring, count in counts.items(): # Iterate through the measurement outcomes (bitstrings) and their counts.
            # The bitstring is ordered from num_qubits-1 down to 0 in Qiskit's counts. We reverse it to match qubit indexing (qubit 0 is at the start).
            reversed_bitstring = bitstring[::-1]
            for i in range(num_qubits): # Loop through each qubit index.
                outcome = int(reversed_bitstring[i]) # Get the measurement outcome (0 or 1) for this qubit in this bitstring.
                if outcome not in qubit_counts[i]: # If we haven't seen this outcome for this qubit yet, initialize its count to 0.
                    qubit_counts[i][outcome] = 0
                qubit_counts[i][outcome] += count # Increment the count for this outcome for this qubit.
                
        # -=-=-=- Basis Sifting and QBER Calculation -=-=-=-
        matching_bases_indices = []
        errors = 0
        sifted_bits_cnt = 0
        
        measured_counts = qubit_counts[i]
        for i in range(num_qubits): # Loop through each qubit/round.
            if alice_bases[i] == bob_bases[i]: # Check if Alice's and Bob's bases matched for this round.
                sifted_bits_cnt += 1
                matching_bases_indices.append(i)
                
                if 0 in measured_counts and 1 in measured_counts:
                    # If both 0 and 1 were measured for this qubit, find the one with the highest count.
                    most_frequent_outcome = 0 if measured_counts[0] > measured_counts[1] else 1
                elif 0 in measured_counts:
                    # If only 0 was measured for this qubit.
                    most_frequent_outcome = 0
                elif 1 in measured_counts:
                    # If only 1 was measured for this qubit.
                    most_frequent_outcome = 1
                else: # This case should ideally not happen if shots > 0 and num_qubits > 0
                    continue
                
                # Noise error
                if alice_bits[i] != most_frequent_outcome:
                    errors += 1
        if sifted_bits_cnt > 0:
            qber = errors / sifted_bits_cnt # QBER is the number of errors divided by the total number of bits in the raw key after sifting.
        else:
            qber = 0.0 # If no bits were sifted, the QBER is 0.0 (no errors in 0 bits).
            
        print(f"\nNumber of qubits sent: {num_qubits}")
        print(f"Number of bits after sifting: {sifted_bits_cnt}")
        print(f"Number of errors after sifting: {errors}")
        print(f"Quantum Bit Error Rate (QBER): {qber:.4f}")
    
        # -=-=-=- Estimate Secure Key Length (Theoretical Formula) -=-=-=-
        def binary_entropy(p): # Define the binary entropy function, h(p) = -p*log2(p) - (1-p)*log2(1-p).
            if p == 0 or p == 1:
                return 0
            return -p * np.log2(p) - (1 - p) * np.log2(1 - p) # The formula for binary entropy.

        if qber <= 0.11: # Approximate theoretical threshold for BB84 (11% is a common value).
            secure_key_rate_per_sifted_bit = 1 - 2 * binary_entropy(qber) # Calculate the rate using the formula R = 1 - 2*h(QBER).
            estimated_secure_key_length = secure_key_rate_per_sifted_bit * sifted_bits_cnt # Estimate the total length of the secure key.
            print(f"Estimated Secure Key Rate (per sifted bit): {secure_key_rate_per_sifted_bit:.4f}")
            print(f"Estimated Secure Key Length: {estimated_secure_key_length:.2f}")
        else:
            print("QBER is too high to generate a secure key.")
            
    # Store QBER results
    if sifted_bits_cnt > 0:
        qber = errors / sifted_bits_cnt
        qber_results.append(qber)
        print(f"QBER for p_depol = {p_depol:.2f}: {qber:.4f}")

# At the end, print summary of results
print("\n-=-=-=- Summary of Results -=-=-=-")
for p, qber in zip(p_depol_values, qber_results):
    print(f"p_depol = {p:.2f}: QBER = {qber:.4f}")
